In [1]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 1.8MB 5.9MB/s 
     |████████████████████████████████| 890kB 47.8MB/s 
     |████████████████████████████████| 3.2MB 54.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=9a78a1a1ee0a2b301b151e7f0fa1783d69294fdd820f28e785074befdf105ac5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 184kB 4.3MB/s 
     |████████████████████████████████| 245kB 23.7MB/s 
     |████████████████████████████████| 102kB 11.0MB/s 
     |████████████████████████████████| 20.7MB 1.6MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [2]:
from datasets import load_dataset
dataset = load_dataset('turkish_product_reviews', split = "train[:10%]")

Using custom data configuration default


Dataset turkish_product_reviews downloaded and prepared to /root/.cache/huggingface/datasets/turkish_product_reviews/default/1.0.0/d1cbeaf7a7a02807d8e59d4af128f3186586e7e3f146c444e7e9e3ed38906511. Subsequent calls will reuse this data.


In [3]:
from transformers import AutoTokenizer, AutoModel

PRE_TRAINED_MODEL_NAME = 'savasy/bert-base-turkish-sentiment-cased'

tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
model = AutoModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [4]:
dataset

Dataset({
    features: ['sentence', 'sentiment'],
    num_rows: 23516
})

In [5]:
dataset[0]["sentence"]

'beklentimin altında bir ürün kaliteli değil'

In [6]:
def read_split(split_dir):
    texts = []
    labels = []
    #values_dataset = list(dataset.values())
    for i in dataset:
      texts.append(i["sentence"])
      labels.append(i["sentiment"])

    return texts, labels

In [7]:
texts, labels = read_split(dataset)

In [8]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=.2)

In [9]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [10]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [11]:
import torch

class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ReviewDataset(train_encodings, train_labels)
val_dataset = ReviewDataset(val_encodings, val_labels)
test_dataset = ReviewDataset(test_encodings, test_labels)

In [12]:
from torch.utils.data import DataLoader
from transformers import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = AutoModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = outputs[0]
        loss.sum().backward()
        optim.step()

model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [13]:
torch.save(model, "/content/drive/MyDrive/Colab_Notebooks/pytorch_model")

In [17]:
review_text = "Bu ürünü çok sevdim"
encoded_review = tokenizer.encode_plus(review_text)

In [18]:
input_ids = encoded_review['input_ids']
attention_mask = encoded_review['attention_mask']